In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler

data = pd.read_csv(r'C:\Users\Kerry\spend_analysis_dataset.csv')
print(data.shape)
data.head()

(500, 9)


,TransactionID,ItemName,Category,Quantity,UnitPrice,TotalCost,PurchaseDate,Supplier,Buyer
0,TXN001,Desk Chair,Furniture,10,113.15,1131.50,2024-04-19,TechMart Inc.,Kelly Joseph
1,TXN002,Stapler,Office Supplies,16,12.62,201.92,2024-07-06,CloudSoft Corp.,Kelly Joseph
2,TXN003,Annual Software License,Software,1,5649.34,5649.34,2024-09-10,TechMart Inc.,Kelly Joseph
3,TXN004,Notepad,Stationery,13,2.92,37.96,2024-01-21,FurniWorks Ltd.,Luis Holland
4,TXN005,Notepad,Stationery,19,1.39,26.41,2024-02-03,TechMart Inc.,Cynthia Jenkins


In [2]:
data.isnull().sum()

TransactionID    0
ItemName         0
Category         0
Quantity         0
UnitPrice        0
TotalCost        0
PurchaseDate     0
Supplier         0
Buyer            0
dtype: int64

In [3]:
data.duplicated().sum()

0

In [4]:
data.describe

<bound method NDFrame.describe of     TransactionID                 ItemName         Category  Quantity  \
0          TXN001               Desk Chair        Furniture        10   
1          TXN002                  Stapler  Office Supplies        16   
2          TXN003  Annual Software License         Software         1   
3          TXN004                  Notepad       Stationery        13   
4          TXN005                  Notepad       Stationery        19   
..            ...                      ...              ...       ...   
495        TXN496               Whiteboard        Furniture        19   
496        TXN497                   Laptop      Electronics         5   
497        TXN498                  Notepad       Stationery         3   
498        TXN499               Whiteboard        Furniture        17   
499        TXN500                  Stapler  Office Supplies         4   

     UnitPrice  TotalCost PurchaseDate            Supplier            Buyer  
0       113

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   TransactionID  500 non-null    object 
 1   ItemName       500 non-null    object 
 2   Category       500 non-null    object 
 3   Quantity       500 non-null    int64  
 4   UnitPrice      500 non-null    float64
 5   TotalCost      500 non-null    float64
 6   PurchaseDate   500 non-null    object 
 7   Supplier       500 non-null    object 
 8   Buyer          500 non-null    object 
dtypes: float64(2), int64(1), object(6)
memory usage: 35.3+ KB
